In [ ]:
cd ..

In [ ]:
import os

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import h5py

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
from echofilter.rawloader import *

In [ ]:
root_dir = '/media/scott/scratch/Datasets/dsforce/surveyExports'

In [ ]:
def load_transect_data(survey, transect_name, root_dir):

    dirname = os.path.join(
        root_dir,
        'Survey' + str(survey)
    )
    raw_fname = os.path.join(dirname, 'Survey{}_{}_Sv_raw.csv'.format(survey, transect_name))
    bot_fname = os.path.join(dirname, 'Survey{}_{}_bottom.evl'.format(survey, transect_name))
    top_fname = os.path.join(dirname, 'Survey{}_{}_turbulence.evl'.format(survey, transect_name))

    timestamps, depths, signals = transect_loader(raw_fname)
    t_bot, d_bot = evl_loader(bot_fname)
    t_top, d_top = evl_loader(top_fname)

    return timestamps, depths, signals, np.interp(timestamps, t_top, d_top), np.interp(timestamps, t_bot, d_bot)

In [ ]:
def plot_transect_data(survey, transect_name, root_dir):

    timestamps, depths, signals, d_top, d_bot = load_transect_data(survey, transect_name, root_dir)

    plt.figure(figsize=(12, 12))
    plt.pcolormesh(timestamps, -depths, signals.T)
    plt.plot(timestamps, -d_bot, 'b')
    plt.plot(timestamps, -d_top, 'c')
    plt.show()

In [ ]:
survey = 17
transect_name = 'GR1_N0A_E'

plot_transect_data(survey, transect_name, root_dir)

In [ ]:
def get_partition_data(partition, root_dir, partitioning_version='firstpass'):

    dirname = os.path.join(root_dir, 'sets', partitioning_version)
    fname_partition = os.path.join(dirname, partition + '.txt')
    fname_header = os.path.join(dirname, 'header' + '.txt')

    with open(fname_header, 'r') as hf:
        for row in csv.reader(hf):
            header = [entry.strip() for entry in row]
            break

    df = pd.read_csv(fname_partition, header=None, names=header)
    return df

In [ ]:
get_partition_data('train', root_dir)

In [ ]:
def get_partition_file_list(partition, root_dir, include_root=True, partitioning_version='firstpass'):
    df = get_partition_data(partition, root_dir, partitioning_version=partitioning_version)
    fnames = df['Filename']
    fnames = [os.path.join(f.split('_')[0], f.strip()) for f in fnames]
    if include_root:
        fnames = [os.path.join(root_dir, f) for f in fnames]
    return fnames

In [ ]:
get_partition_file_list('train', root_dir)

In [ ]:
print('{:<30s} {:>6s} {:>6s}'.format('Filename', 'Min D', 'Max D'))

for fname in get_partition_file_list('train', root_dir):
    fname = fname.replace('Sv_raw.csv', 'bottom.evl')
    try:
        depths = evl_loader(fname)[1]
    except:
        continue
    print('{:<30s} {:6.1f} {:6.1f}'.format(os.path.split(fname)[1], min(depths), max(depths)))

In [ ]:
survey = 17
transect_name = 'GR4_S3A_E'

plot_transect_data(survey, transect_name, root_dir)

In [ ]:
print('{:<30s} {:>6s} {:>6s}'.format('Filename', 'Min D', 'Max D'))

for fname in get_partition_file_list('validate', root_dir):
    fname = fname.replace('Sv_raw.csv', 'bottom.evl')
    try:
        depths = evl_loader(fname)[1]
    except:
        continue
    print('{:<30s} {:6.1f} {:6.1f}'.format(os.path.split(fname)[1], min(depths), max(depths)))

In [ ]:
survey = 17
transect_name = 'GR1_S3W_F'

plot_transect_data(survey, transect_name, root_dir)

In [ ]:
print('{:<30s} {:>6s} {:>6s}'.format('Filename', 'Min D', 'Max D'))

for fname in get_partition_file_list('test', root_dir):
    fname = fname.replace('Sv_raw.csv', 'bottom.evl')
    try:
        depths = evl_loader(fname)[1]
    except:
        continue
    print('{:<30s} {:6.1f} {:6.1f}'.format(os.path.split(fname)[1], min(depths), max(depths)))

In [ ]:
print('{:<30s} {:>6s} {:>6s}'.format('Filename', 'Min D', 'Max D'))

for fname in get_partition_file_list('leaveout', root_dir):
    fname = fname.replace('Sv_raw.csv', 'bottom.evl')
    try:
        depths = evl_loader(fname)[1]
    except:
        continue
    print('{:<30s} {:6.1f} {:6.1f}'.format(os.path.split(fname)[1], min(depths), max(depths)))

In [ ]:
40, 62, 96

In [ ]:
with h5py.File("mytestfile.hdf5", "w") as f:
    f.create_dataset("mydataset", (100,), dtype='i')

In [ ]:
survey = 17
transect_name = 'GR1_S3W_F'

plot_transect_data(survey, transect_name, root_dir)